In [2]:
import pandas as pd
import pm4py
from pm4py.objects.conversion.log import converter as xes_converter
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.objects.log.obj import EventLog, Trace, Event
import math
import csv
from bidict import bidict
from datetime import datetime, timedelta
import numpy as np
import itertools
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.models import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization
from pre_process import pre_processing
from pre_process import preprocess_graph_attributes
from utility import utility_functions
from utility import constants
from mappers import mappers
from feature_encoding import vectorize_features
from train_and_test import train
from train_and_test import prediction
from evaluation import post_processing
from evaluation import evaluation

In [3]:
# Load and convert the dataset 


# Import the required attributes from the CSV file
df = pd.read_csv('.//logistic_CSS9D_CSS9E.csv')[constants.required_attributes+constants.event_attribute_features+constants.case_attribute+constants.src_dest_attributes]

# Convert the DataFrame into the event log structure
log = xes_converter.apply(df, variant=xes_converter.Variants.TO_EVENT_LOG) 

# Convert the timestamp from string to datetime format
log = pre_processing.convert_timestamp(log)

# Adjust the arrived time
log = pre_processing.adjust_arrived_time(log)

In [4]:

"""
global variables using the entire event log
"""
# Calculate the average dwell time in seconds: 245.68552766191863
average_dwell_time_in_sec = utility_functions.average_dwell_time(log)

# Calculate the average time since case start in seconds: 5737.639253954513
average_time_since_start_in_sec = utility_functions.average_time_since_case_start(log)

# Build the mapper for average dwell time based on EQTYP: 
dwell_time_mapper = mappers.build_average_dwell_time_mapper(log)

In [5]:
"""
Fold creation
"""
# Create folds for training and testing
folds = pre_processing.create_folds(log, 3)

# get the training log from first two folds
training_log = EventLog([case for sub_log in folds[:2] for case in sub_log])

# Use the third fold for testing
testing_log = EventLog([case for sub_log in folds[2:] for case in sub_log])

In [6]:
"""
Global variables using only the training log
"""
# Build the mapper for EID based on the training log
eid_mapper = mappers.build_eid_mapper(training_log)

# Get the maximum case length in the training log and add an end signal
max_case_len = max([len(case) for case in training_log]) + 1

# Build mappers for event attribute features based on the training log
event_attribute_mappers = {attr: mappers.build_event_attr_mapper(training_log, attr) for attr in constants.event_attribute_features}

# Build mappers for case attribute features based on the training log
case_attribute_mappers = {attr: mappers.build_case_attr_mapper(training_log, attr) for attr in constants.case_attribute_features}

# Get the routing netwrok of logistic log
network = preprocess_graph_attributes.preprocess_graph_attributes('.//20231219-subgraph_CSS9D_CSS9E.gml')

In [7]:
# Train a model with training log


# Vectorize the features for training
X = vectorize_features.vectorize_features(training_log, average_time_since_start_in_sec, eid_mapper, dwell_time_mapper, event_attribute_mappers, case_attribute_mappers, max_case_len, network)

# Vectorize the expected EID predictions for training
equipment_prediction = vectorize_features.vectorize_equipment_prediction(training_log, eid_mapper) 

# Vectorize the expected dwell time predictions for training
time_prediction = vectorize_features.vectorize_time_prediction(training_log, dwell_time_mapper) 


In [ ]:
# Train the model using the vectorized features and predictions
train.train(X, equipment_prediction, time_prediction, max_case_len, constants.models_path, num_features=len(X[0][0]),number_of_eids=len(eid_mapper))

Build model...
Metal device set to: Apple M1


2024-02-22 15:48:25.378549: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-22 15:48:25.379408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/500


/Users/sumyea/Desktop/Study/Study/WS21-22/Introduction_to_Data_Science/Assignment/Part1/env/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/nadam.py:86: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-02-22 15:48:25.865258: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-22 15:48:27.362219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-22 15:48:27.775166: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-22 15:48:28.020165: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-22 15:48:28.032389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GP

13/13 - 7s - loss: 6.3388 - equipment_output_loss: 4.4760 - time_output_loss: 1.8628 - val_loss: 4.7859 - val_equipment_output_loss: 4.3788 - val_time_output_loss: 0.4070 - lr: 0.0020 - 7s/epoch - 562ms/step
Epoch 2/500
13/13 - 1s - loss: 5.4348 - equipment_output_loss: 3.8476 - time_output_loss: 1.5871 - val_loss: 4.7838 - val_equipment_output_loss: 4.2976 - val_time_output_loss: 0.4863 - lr: 0.0020 - 1s/epoch - 104ms/step
Epoch 3/500
13/13 - 1s - loss: 4.9598 - equipment_output_loss: 3.4830 - time_output_loss: 1.4768 - val_loss: 4.4999 - val_equipment_output_loss: 4.0128 - val_time_output_loss: 0.4872 - lr: 0.0020 - 1s/epoch - 100ms/step
Epoch 4/500
13/13 - 1s - loss: 4.6228 - equipment_output_loss: 3.1275 - time_output_loss: 1.4953 - val_loss: 4.1082 - val_equipment_output_loss: 3.7239 - val_time_output_loss: 0.3843 - lr: 0.0020 - 1s/epoch - 96ms/step
Epoch 5/500
13/13 - 1s - loss: 4.4159 - equipment_output_loss: 2.9553 - time_output_loss: 1.4606 - val_loss: 4.0324 - val_equipment_o

In [ ]:
# Test the model with testing log


from keras.models import load_model
# Load the trained model
model = load_model(constants.models_path+'model_83-1.42.h5')

# Set the starting prefix size for prediction
starting_prefix_size = 2

# Perform predictions on the testing log using the loaded model
all_prediction = prediction.predict(testing_log, model, average_time_since_start_in_sec, eid_mapper, dwell_time_mapper, event_attribute_mappers, case_attribute_mappers, max_case_len, network, starting_prefix_size)


In [ ]:

# Create a prefix log based on the testing log with a specified prefix size
prefix_log = post_processing.create_prefix_log(testing_log, starting_prefix_size)

# Perform post-processing on the prefix log using the predicted results
prediction_log = post_processing.post_process(prefix_log, all_prediction)

# Add ground truth information to the predicted events in the prediction log
prediction_log = post_processing.add_ground_truth(testing_log, prediction_log)

# Convert the prediction log to a DataFrame and save it to a CSV file
result_with_ground_truth = xes_converter.apply(prediction_log, variant=xes_converter.Variants.TO_DATA_FRAME)
pm4py.write_xes(log, 'result_log.xes')
result_with_ground_truth.to_csv('result.csv')
# Calculate the accuracy of EID prediction
accuracy_of_equipment_prediction = evaluation.calculate_accuracy(result_with_ground_truth, len(prediction_log), starting_prefix_size)

# Calculate MAE and MAPE for dwell time prediction
error_mappers = evaluation.calculate_mae_mape(prediction_log, starting_prefix_size)

# Print the MAE and MAPE for timestamp prediction
# print("MAE for next timestamp prediction")
# print(error_mappers['MAE'])
# print("MAPE for next timestamp prediction")
# print(error_mappers['MAPE'])